<a href="https://colab.research.google.com/github/paulxuereb/ML/blob/master/Seq2SeqAllenNLP_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.realworldnlpbook.com/blog/building-seq2seq-machine-translation-models-using-allennlp.html

In [1]:
!pip install AllenNLP

    100% |████████████████████████████████| 5.6MB 8.8MB/s 
    100% |████████████████████████████████| 122kB 37.5MB/s 
    100% |████████████████████████████████| 1.5MB 25.6MB/s 
    100% |████████████████████████████████| 583kB 31.6MB/s 
    100% |████████████████████████████████| 51kB 28.9MB/s 
    100% |████████████████████████████████| 245kB 35.2MB/s 
    100% |████████████████████████████████| 245kB 34.1MB/s 
    100% |████████████████████████████████| 133kB 41.6MB/s 
    100% |████████████████████████████████| 51kB 26.5MB/s 
    100% |████████████████████████████████| 5.4MB 10.0MB/s 
    100% |████████████████████████████████| 51kB 29.8MB/s 
    100% |████████████████████████████████| 2.1MB 19.7MB/s 
    100% |████████████████████████████████| 143kB 28.5MB/s 
    100% |████████████████████████████████| 2.3MB 18.1MB/s 
    100% |████████████████████████████████| 92kB 37.3MB/s 
    100% |████████████████████████████████| 92kB 36.4MB/s 
    100% |████████████████████████████████| 20

In [2]:
    
import itertools

import torch
import torch.optim as optim
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


Setup data files

In [0]:
!mkdir data

In [5]:
%cd data

[Errno 2] No such file or directory: 'data'
/content/data


In [4]:
!wget http://downloads.tatoeba.org/exports/sentences.tar.bz2

--2019-04-25 09:48:58--  http://downloads.tatoeba.org/exports/sentences.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.tatoeba.org/exports/sentences.tar.bz2 [following]
--2019-04-25 09:48:58--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118227705 (113M) [application/octet-stream]
Saving to: ‘sentences.tar.bz2’

sentences.tar.bz2   100%[===================>] 112.75M  19.2MB/s    in 6.9s    

2019-04-25 09:49:06 (16.4 MB/s) - ‘sentences.tar.bz2’ saved [118227705/118227705]



In [5]:
!wget http://downloads.tatoeba.org/exports/links.tar.bz2

URL transformed to HTTPS due to an HSTS policy
--2019-04-25 09:49:09--  https://downloads.tatoeba.org/exports/links.tar.bz2
Resolving downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Connecting to downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80800484 (77M) [application/octet-stream]
Saving to: ‘links.tar.bz2’

links.tar.bz2       100%[===================>]  77.06M  19.3MB/s    in 4.8s    

2019-04-25 09:49:15 (16.1 MB/s) - ‘links.tar.bz2’ saved [80800484/80800484]



In [0]:
!ls

In [7]:
!pwd

/content/data


In [6]:
!tar -xvf sentences.tar.bz2

sentences.csv


In [7]:
!tar -xvf links.tar.bz2

links.csv


In [8]:
!wget https://raw.githubusercontent.com/mhagiwara/realworldnlp/master/examples/mt/create_bitext.py

--2019-04-25 09:49:54--  https://raw.githubusercontent.com/mhagiwara/realworldnlp/master/examples/mt/create_bitext.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4870 (4.8K) [text/plain]
Saving to: ‘create_bitext.py’

create_bitext.py    100%[===================>]   4.76K  --.-KB/s    in 0s      

2019-04-25 09:49:54 (88.2 MB/s) - ‘create_bitext.py’ saved [4870/4870]



In [0]:
!python create_bitext.py eng_cmn sentences.csv links.csv \
    | cut -f3,6 > tatoeba.eng_cmn.tsv

In [0]:
!cat tatoeba.eng_cmn.tsv | awk 'NR%10==1' > tatoeba.eng_cmn.test.tsv
!cat tatoeba.eng_cmn.tsv | awk 'NR%10==2' > tatoeba.eng_cmn.dev.tsv
!cat tatoeba.eng_cmn.tsv | awk 'NR%10!=1&&NR%10!=2' > tatoeba.eng_cmn.train.tsv

In [11]:
!ls

create_bitext.py  sample_data		   tatoeba.eng_cmn.test.tsv
data		  sentences.csv		   tatoeba.eng_cmn.train.tsv
links.csv	  sentences.tar.bz2	   tatoeba.eng_cmn.tsv
links.tar.bz2	  tatoeba.eng_cmn.dev.tsv


In [12]:
!head tatoeba.eng_cmn.train.tsv

Today is June 18th and it is Muiriel's birthday!	今天是６月１８号，也是Muiriel的生日！
Muiriel is 20 now.	Muiriel现在20岁了。
The password is "Muiriel".	密码是"Muiriel"。
The password is "Muiriel".	密碼是「Muiriel」。
I will be back soon.	我很快就會回來。
I'm at a loss for words.	我不知道應該說什麼才好。
This is never going to end.	這個永遠完不了了。
This is never going to end.	这将永远继续下去。
That was an evil bunny.	那是一隻有惡意的兔子。
I was in the mountains.	我以前在山里。


Grab the Microsoft chat files

In [13]:
!wget https://qnamakerstore.blob.core.windows.net/qnamakerdata/editorial/qna_chitchat_the_professional.tsv

--2019-04-25 09:52:48--  https://qnamakerstore.blob.core.windows.net/qnamakerdata/editorial/qna_chitchat_the_professional.tsv
Resolving qnamakerstore.blob.core.windows.net (qnamakerstore.blob.core.windows.net)... 13.88.144.240
Connecting to qnamakerstore.blob.core.windows.net (qnamakerstore.blob.core.windows.net)|13.88.144.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62473 (61K) [text/tab-separated-values]
Saving to: ‘qna_chitchat_the_professional.tsv’

qna_chitchat_the_pr 100%[===================>]  61.01K  --.-KB/s    in 0.04s   

2019-04-25 09:52:48 (1.35 MB/s) - ‘qna_chitchat_the_professional.tsv’ saved [62473/62473]



In [0]:
import pandas as pd
df = pd.read_csv("qna_chitchat_the_professional.tsv",sep='\t')
#df.drop(['Source', 'Metadata'])
#df1 = df[['user1','user2']]
#df1

In [0]:
df1 = df[['Question','Answer']]
df1
df1.to_csv('chat.tsv', sep = '\t', header=False, index=False)

In [28]:
!head chat.tsv

What's your age?	Age doesn't really apply to me.
Are you young?	Age doesn't really apply to me.
When were you born?	Age doesn't really apply to me.
What age are you?	Age doesn't really apply to me.
Are you old?	Age doesn't really apply to me.
How old are you?	Age doesn't really apply to me.
How long ago were you born?	Age doesn't really apply to me.
Ask me anything	I'm better at answering questions.
Ask me a question	I'm better at answering questions.
Can you ask me a question?	I'm better at answering questions.


In [0]:
!cat chat.tsv | awk 'NR%10==1' > chat.eng_cmn.test.tsv
!cat chat.tsv | awk 'NR%10==2' > chat.eng_cmn.dev.tsv
!cat chat.tsv | awk 'NR%10!=1&&NR%10!=2' > chat.eng_cmn.train.tsv

In [32]:
!ls
#!head chat.eng_cmn.dev.tsv

Are you young?	Age doesn't really apply to me.
What do you want to know about me?	I'm better at answering questions.
You bore me	I aim for efficiency.
Why are you so boring?	I aim for efficiency.
Cook me something	That's not something I can do.
What can you do?	I'm here to answer your questions and help out.
Which people made you?	People created me.
Do you have a family?	I don't have family.
Are you a man?	That's a biological concept that doesn't apply to me.
What's your gender?	That's a biological concept that doesn't apply to me.


Build the model

In [34]:
 bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True)

100%|██████████| 407873900/407873900 [00:11<00:00, 35359038.76B/s]


In [36]:
EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256
CUDA_DEVICE = 0


reader = Seq2SeqDatasetReader(
        source_tokenizer=WordTokenizer(),
        target_tokenizer=CharacterTokenizer(),
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
train_dataset = reader.read('chat.eng_cmn.train.tsv')
validation_dataset = reader.read('chat.eng_cmn.dev.tsv')

vocab = Vocabulary.from_instances(train_dataset + validation_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3})

in_embedding = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                          allow_unmatched_keys=True)
BERT_DIM = in_embedding.get_output_dim()


encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(BERT_DIM,
                                              HIDDEN_DIM,
                                              batch_first=True))

attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
max_decoding_steps = 250
model = SimpleSeq2Seq(vocab, in_embedding, encoder, max_decoding_steps,
                      target_embedding_dim=BERT_DIM,
                      target_namespace="target_tokens",
                      attention=attention, beam_size=8)


optimizer = optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])

iterator.index_with(vocab)

525it [00:00, 10653.58it/s]
66it [00:00, 7146.06it/s]
100%|██████████| 591/591 [00:00<00:00, 33350.38it/s]


In [0]:
#skip this
en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=EN_EMBEDDING_DIM)
# encoder = PytorchSeq2SeqWrapper(
#     torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)

source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})



# attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())
# attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
attention = DotProductAttention()

max_decoding_steps = 20   # TODO: make this variable
model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)
optimizer = optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])

iterator.index_with(vocab)
#end skip

In [51]:
CUDA_DEVICE = 0

if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
    

trainer = Trainer(model=model,
                      optimizer=optimizer,
                      iterator=iterator,
                      train_dataset=train_dataset,
                      validation_dataset=validation_dataset,
                      num_epochs=1,
                      cuda_device=CUDA_DEVICE)



You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
  0%|          | 0/17 [00:00<?, ?it/s]

Epoch: 0


loss: 0.2050 ||: 100%|██████████| 17/17 [00:01<00:00, 11.31it/s]
BLEU: 0.0189, loss: 0.2451 ||: 100%|██████████| 3/3 [00:00<00:00,  5.71it/s]
loss: 0.1972 ||:   6%|▌         | 1/17 [00:00<00:01,  8.71it/s]

Epoch: 1


loss: 0.2055 ||: 100%|██████████| 17/17 [00:01<00:00, 11.04it/s]
BLEU: 0.0154, loss: 0.2138 ||: 100%|██████████| 3/3 [00:00<00:00,  5.68it/s]
  0%|          | 0/17 [00:00<?, ?it/s]

...

Epoch: 48


loss: 0.1908 ||: 100%|██████████| 17/17 [00:01<00:00, 11.48it/s]
BLEU: 0.0053, loss: 0.2511 ||: 100%|██████████| 3/3 [00:00<00:00,  5.88it/s]
loss: 0.1219 ||:   6%|▌         | 1/17 [00:00<00:01,  9.09it/s]

Epoch: 49


loss: 0.1906 ||: 100%|██████████| 17/17 [00:01<00:00, 11.02it/s]
BLEU: 0.0445, loss: 0.2035 ||: 100%|██████████| 3/3 [00:00<00:00,  6.05it/s]


SOURCE: [@start@, Are, you, young, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', 'a', 'y', '.']
SOURCE: [@start@, What, do, you, want, to, know, about, me, ?, @end@]
GOLD: [@start@, I, ', m,  , b, e, t, t, e, r,  , a, t,  , a, n, s, w, e, r, i, n, g,  , q, u, e, s, t, i, o, n, s, ., @end@]
PRED: ['I', ' ', 'w', 'o', 'u', 'l', 'd', 'n', "'", 't', ' ', 'k', 'n', 'o', 'w', ' ', 'h', 'o', 'w', ' ', 't', 'o', ' ', 'a', 'd', 'v', 'i', 's', 'e', ' ', 'a', 'b', 'o', 'u', 't', ' ', 't', 'h', 'i', 's', '.']
SOURCE: [@start@, You, bore, me, @end@]
GOLD: [@start@, I,  , a, i, m,  , f, o, r,  , e, f, f, i, c, i, e, n, c, y, ., @end@]
PRED: ['I', ' ', 'a', 'i', 'm', ' ', 't', 'o', ' ', 's', 'e', 'r', 'v', 'e', '.']
SOURCE: [@start@, Why, are, you, so, boring, ?, @end@]
GOLD: [@start@, I,  , a, i, m,  , f, o, r,  , e, f, f, i, c, i, e, n, c, y, ., @end@]
PRED: ['O', 'k', '.']
SOURCE: [@start@, Cook, me, s

In [53]:
for i in range(50):
    print('Epoch: {}'.format(i))
    metrics = trainer.train()

    predictor = SimpleSeq2SeqPredictor(model, reader)

    #do predictions on last epoch
    if i >= 49:
      for instance in itertools.islice(validation_dataset, 10):
          print('SOURCE:', instance.fields['source_tokens'].tokens)
          print('GOLD:', instance.fields['target_tokens'].tokens)
          print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])
          
metrics

loss: 0.1814 ||:   6%|▌         | 1/17 [00:00<00:01,  9.98it/s]

Epoch: 0


loss: 0.1919 ||: 100%|██████████| 17/17 [00:01<00:00, 11.16it/s]
BLEU: 0.0643, loss: 0.2168 ||: 100%|██████████| 3/3 [00:00<00:00,  5.96it/s]
  0%|          | 0/17 [00:00<?, ?it/s]

Epoch: 1


loss: 0.1911 ||: 100%|██████████| 17/17 [00:01<00:00, 11.61it/s]
BLEU: 0.0436, loss: 0.2139 ||: 100%|██████████| 3/3 [00:00<00:00,  6.15it/s]
loss: 0.1888 ||:   6%|▌         | 1/17 [00:00<00:01,  9.18it/s]

...

loss: 0.1858 ||: 100%|██████████| 17/17 [00:01<00:00, 11.23it/s]
BLEU: 0.0102, loss: 0.2098 ||: 100%|██████████| 3/3 [00:00<00:00,  6.14it/s]
  0%|          | 0/17 [00:00<?, ?it/s]

Epoch: 49


loss: 0.1847 ||: 100%|██████████| 17/17 [00:01<00:00, 11.40it/s]
BLEU: 0.1109, loss: 0.2134 ||: 100%|██████████| 3/3 [00:00<00:00,  5.96it/s]


SOURCE: [@start@, Are, you, young, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', 'a', 'y', '.']
SOURCE: [@start@, What, do, you, want, to, know, about, me, ?, @end@]
GOLD: [@start@, I, ', m,  , b, e, t, t, e, r,  , a, t,  , a, n, s, w, e, r, i, n, g,  , q, u, e, s, t, i, o, n, s, ., @end@]
PRED: ['V', 'e', 'r', 'y', ' ', 'w', 'e', 'l', 'l', '.']
SOURCE: [@start@, You, bore, me, @end@]
GOLD: [@start@, I,  , a, i, m,  , f, o, r,  , e, f, f, i, c, i, e, n, c, y, ., @end@]
PRED: ['I', ' ', 'a', 'p', 'o', 'l', 'o', 'g', 'i', 'z', 'e', '.']
SOURCE: [@start@, Why, are, you, so, boring, ?, @end@]
GOLD: [@start@, I,  , a, i, m,  , f, o, r,  , e, f, f, i, c, i, e, n, c, y, ., @end@]
PRED: ['I', ' ', 'd', 'o', 'n', "'", 't', ' ', 'h', 'a', 'v', 'e', ' ', 'f', 'a', 'm', 'i', 'l', 'y', '.']
SOURCE: [@start@, Cook, me, something, @end@]
GOLD: [@start@, T, h, a, t, ', s,  , n, o, t,  , s, o, m, e, t, h, i

{'best_epoch': 63,
 'best_validation_BLEU': 0.08688639690992632,
 'best_validation_loss': 0.20112804571787515,
 'epoch': 0,
 'peak_cpu_memory_MB': 3659.152,
 'peak_gpu_0_memory_MB': 1371,
 'training_cpu_memory_MB': 3659.152,
 'training_duration': '00:00:02',
 'training_epochs': 0,
 'training_gpu_0_memory_MB': 1371,
 'training_loss': 0.1846960541956565,
 'training_start_epoch': 0,
 'validation_BLEU': 0.11087748155630821,
 'validation_loss': 0.2134461055199305}

In [54]:
for instance in itertools.islice(train_dataset, 50):
          print('SOURCE:', instance.fields['source_tokens'].tokens)
          print('GOLD:', instance.fields['target_tokens'].tokens)
          print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])

SOURCE: [@start@, When, were, you, born, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', '.']
SOURCE: [@start@, What, age, are, you, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', '.']
SOURCE: [@start@, Are, you, old, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', 'a', 'y', '.']
SOURCE: [@start@, How, old, are, you, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', '.']
SOURCE: [@start@, How, long, ago, were, you, born, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['O', 'k', 'a', 'y', ',', ' ', 'b', 'u', 't', ' ', 'I', "'", 'm'

Seq2Seq with attention (non BERT)

In [64]:
EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256
CUDA_DEVICE = 0


reader = Seq2SeqDatasetReader(
        source_tokenizer=WordTokenizer(),
        target_tokenizer=CharacterTokenizer(),
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
train_dataset = reader.read('chat.eng_cmn.train.tsv')
validation_dataset = reader.read('chat.eng_cmn.dev.tsv')

vocab = Vocabulary.from_instances(train_dataset + validation_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3})

en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=EN_EMBEDDING_DIM)
# encoder = PytorchSeq2SeqWrapper(
#     torch.nn.LSTM(EN_EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)

source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})



# attention = LinearAttention(HIDDEN_DIM, HIDDEN_DIM, activation=Activation.by_name('tanh')())
# attention = BilinearAttention(HIDDEN_DIM, HIDDEN_DIM)
attention = DotProductAttention()

max_decoding_steps = 100   # TODO: make this variable
model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)
optimizer = optim.Adam(model.parameters())
iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])

iterator.index_with(vocab)

CUDA_DEVICE = 0

if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
    

trainer = Trainer(model=model,
                      optimizer=optimizer,
                      iterator=iterator,
                      train_dataset=train_dataset,
                      validation_dataset=validation_dataset,
                      num_epochs=1,
                      cuda_device=CUDA_DEVICE)

525it [00:00, 2573.64it/s]
66it [00:00, 6548.33it/s]
100%|██████████| 591/591 [00:00<00:00, 38605.10it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


In [65]:
for i in range(50):
    print('Epoch: {}'.format(i))
    metrics = trainer.train()

    predictor = SimpleSeq2SeqPredictor(model, reader)

    #do predictions on last epoch
    if i >= 49:
      for instance in itertools.islice(validation_dataset, 10):
          print('SOURCE:', instance.fields['source_tokens'].tokens)
          print('GOLD:', instance.fields['target_tokens'].tokens)
          print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])
          
metrics

loss: 3.9548 ||:   6%|▌         | 1/17 [00:00<00:01,  8.84it/s]

Epoch: 0


loss: 3.2644 ||: 100%|██████████| 17/17 [00:01<00:00, 15.24it/s]
BLEU: 0.0000, loss: 3.0252 ||: 100%|██████████| 3/3 [00:00<00:00,  3.66it/s]
  0%|          | 0/17 [00:00<?, ?it/s]

Epoch: 1


loss: 2.9775 ||: 100%|██████████| 17/17 [00:01<00:00, 15.86it/s]
BLEU: 0.0000, loss: 2.8850 ||: 100%|██████████| 3/3 [00:00<00:00,  3.87it/s]
  0%|          | 0/17 [00:00<?, ?it/s]

...

loss: 0.1482 ||: 100%|██████████| 17/17 [00:01<00:00, 15.55it/s]
BLEU: 0.1932, loss: 0.3435 ||: 100%|██████████| 3/3 [00:00<00:00,  4.11it/s]
loss: 0.1278 ||:  12%|█▏        | 2/17 [00:00<00:01, 13.57it/s]

Epoch: 49


loss: 0.1343 ||: 100%|██████████| 17/17 [00:01<00:00, 15.71it/s]
BLEU: 0.2314, loss: 0.3391 ||: 100%|██████████| 3/3 [00:00<00:00,  4.58it/s]


SOURCE: [@start@, Are, you, young, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: ['I', "'", 'm', ' ', 'd', 'i', 'g', 'i', 't', 'a', 'l', '.']
SOURCE: [@start@, What, do, you, want, to, know, about, me, ?, @end@]
GOLD: [@start@, I, ', m,  , b, e, t, t, e, r,  , a, t,  , a, n, s, w, e, r, i, n, g,  , q, u, e, s, t, i, o, n, s, ., @end@]
PRED: ['V', 'e', 'r', 'y', ' ', 'w', 'e', 'l', 'l', '.']
SOURCE: [@start@, You, bore, me, @end@]
GOLD: [@start@, I,  , a, i, m,  , f, o, r,  , e, f, f, i, c, i, e, n, c, y, ., @end@]
PRED: ['I', ' ', 'a', 'p', 'o', 'l', 'o', 'g', 'i', 'z', 'e', '.']
SOURCE: [@start@, Why, are, you, so, boring, ?, @end@]
GOLD: [@start@, I,  , a, i, m,  , f, o, r,  , e, f, f, i, c, i, e, n, c, y, ., @end@]
PRED: ['I', ' ', 'd', 'o', 'n', "'", 't', ' ', 'h', 'a', 'v', 'e', ' ', 'f', 'a', 'm', 'i', 'l', 'y', '.']
SOURCE: [@start@, Cook, me, something, @end@]
GOLD: [@start@, T, h, a, t, ', s,

{'best_epoch': 46,
 'best_validation_BLEU': 0.17964834325228898,
 'best_validation_loss': 0.3206319808959961,
 'epoch': 0,
 'peak_cpu_memory_MB': 3669.104,
 'peak_gpu_0_memory_MB': 1371,
 'training_cpu_memory_MB': 3669.104,
 'training_duration': '00:00:01',
 'training_epochs': 0,
 'training_gpu_0_memory_MB': 1371,
 'training_loss': 0.13425707466462078,
 'training_start_epoch': 0,
 'validation_BLEU': 0.2313778655724189,
 'validation_loss': 0.3390659987926483}

In [73]:
for instance in itertools.islice(train_dataset, 10):
    print('SOURCE:', instance.fields['source_tokens'].tokens)
    print('GOLD:', instance.fields['target_tokens'].tokens)
    print('PRED:', ''.join(predictor.predict_instance(instance)['predicted_tokens']))

SOURCE: [@start@, When, were, you, born, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: Got it.
SOURCE: [@start@, What, age, are, you, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: I don't have a body.
SOURCE: [@start@, Are, you, old, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: I'm digital.
SOURCE: [@start@, How, old, are, you, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: Great, thanks.
SOURCE: [@start@, How, long, ago, were, you, born, ?, @end@]
GOLD: [@start@, A, g, e,  , d, o, e, s, n, ', t,  , r, e, a, l, l, y,  , a, p, p, l, y,  , t, o,  , m, e, ., @end@]
PRED: Great, thanks.
SOURCE: [@start@, Ask, me, anything, @end@]
GOLD: [@start@, I, ', 

In [70]:
''.join(predictor.predict_instance(instance)['predicted_tokens'])

"I don't have family."